In [ ]:
!pip install dash
!pip install pandas

In [34]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [45]:
app = dash.Dash(__name__)


In [46]:
df = pd.read_csv("intro_bees.csv")

df = df.groupby(['State','ANSI','Affected by', 'Year', 'state_code']) [['Pct of Colonies Impacted']].mean()
df.reset_index(inplace = True)
print(df[:5])

     State  ANSI Affected by  Year state_code  Pct of Colonies Impacted
0  Alabama     1     Disease  2015         AL                      0.05
1  Alabama     1     Disease  2016         AL                      1.20
2  Alabama     1     Disease  2017         AL                      2.25
3  Alabama     1     Disease  2018         AL                      1.30
4  Alabama     1     Disease  2019         AL                      1.80


In [47]:
app.layout = html.Div([
    html.H1("Aplicação web com dash", style={'text-align':'center', 'font-family': 'verdana'}),

    dcc.Dropdown(id = "slct_year",
                 options=[
                    {"label":"2015","value":2015},
                    {"label":"2016","value":2016},
                    {"label":"2017","value":2017},
                    {"label":"2018","value":2018}],
                 multi = False,
                 value = 2015,
                 style = {"width":"40%"}
                 ),
    html.Div(id = 'output_container', children= []),
    html.Br(),

    dcc.Graph(id = 'my_bee_map', figure = {})
])

In [48]:
@app.callback(
    [Output(component_id = 'output_container', component_property = 'children'),
     Output(component_id = 'my_bee_map', component_property = 'figure')],
    [Input(component_id = 'slct_year', component_property = 'value')]
)

def update_graph(option_slctd):
  print(option_slctd)
  print(type(option_slctd))

  container = "The year chosen by user was: {}". format(option_slctd)

  dfc = df.copy()
  dfc = dfc[dfc["Year"] == option_slctd]
  dfc = dfc[dfc["Affected by"] == "Varroa_mites"]

  fig = px.choropleth(
    data_frame = dfc,
    locationmode = 'USA-states',
    locations = 'state_code',
    scope = "usa",
    color = "Pct of Colonies Impacted",
    hover_data = ['State','Pct of Colonies Impacted'],
    color_continuous_scale= px.colors.sequential.YlOrRd,
    labels = {'Pct of Colonies Impacted', '% of Bee Colonies'},
    template = "plotly_dark"
)

  return container, fig

In [49]:
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
#  Plotly Graph Objects (GO)
#     fig = go.Figure(
#         data=[go.Choropleth(
#             locationmode='USA-states',
#             locations=dff['state_code'],
#             z=dff["Pct of Colonies Impacted"].astype(float),
#             colorscale='Reds',
#         )]
#     )

#     fig.update_layout(
#         title_text="Bees Affected by Mites in the USA",
#         title_xanchor="center",
#         title_font=dict(size=24),
#         title_x=0.5,
#         geo=dict(scope='usa'),
#     )